# Michelson Interferometer

## Exp 1: 이동 거리 측정
$2d = m \lambda$

In [1]:
import os
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
def error_rate(real: float, exp: float):
    return 100*abs(real-exp)/real

In [3]:
wavelength = 632.8   # He-Ne laser wavelength
base_path = "./exp5_data/"
distance_data = []
with open(os.path.join(base_path, "distance.txt"), "r") as f:
    while True:
        try:
            d_n, V = f.readline().split()
            distance_data.append((int(d_n), float(V)))
        except:
            break
            
def get_distance(d_n: int, V: float):
    d_n, V = int(d_n), float(V)
    distance = d_n * wavelength / 2
    actuator_distance = distance/V
    # print(f"Moving Distance per {V:.4f}V: {distance:.4f}nm")
    # print(f"Moving Distance per 1V: {(actuator_distance):.4f}nm")
    return actuator_distance

all_distance = []
for d_n, V in distance_data:
    distance = get_distance(d_n, V)
    print(f"Actuator Distance: {distance:.4f}nm/V")
    all_distance.append(distance)
actuator_distance = np.mean(all_distance)
print(f"Mean Act. Dist.: {actuator_distance:.4f}nm/V")

Actuator Distance: 119.5466nm/V
Actuator Distance: 120.9172nm/V
Actuator Distance: 121.1745nm/V
Mean Act. Dist.: 120.5461nm/V


## Exp 2: 미지 광원

In [16]:
unknown_data = []
with open(os.path.join(base_path, "unknown.txt"), "r") as f:
    while True:
        try:
            d_n, V = f.readline().split()
            unknown_data.append((int(d_n), float(V)))
        except:
            break

In [17]:
# Green Wavelength at light-adapted eye
green_light = 555
def get_wavelength(d_n: int, V: float, actuator_distance: float):
    avg_distance = actuator_distance * V
    wavelength = 2 * avg_distance / d_n
    print(f"Wavelength : {wavelength:.4f}nm")
    return wavelength

all_wavlength = []
for d_n, V in unknown_data:
    wavelength = get_wavelength(d_n, V, actuator_distance)
    all_wavlength.append(wavelength)
    print(f"Error Rate: {error_rate(green_light, wavelength):.4f}%")
mean_wavelength = np.mean(all_wavlength)
print(f"Mean Wavelength: {mean_wavelength:.4f}nm")
print(f"Error Rate: {error_rate(green_light, mean_wavelength)}")

Wavelength : 536.4301nm
Error Rate: 3.3459%
Wavelength : 472.5407nm
Error Rate: 14.8575%
Wavelength : 539.6447nm
Error Rate: 2.7667%
Wavelength : 535.5260nm
Error Rate: 3.5088%
Wavelength : 533.2959nm
Error Rate: 3.9106%
Mean Wavelength: 523.4875nm
Error Rate: 5.677934874260622


## Exp 3: 공기 굴절률
$n = \cfrac{\lambda}{2LC} P + 1$  
$\Delta{P} = C{\Delta}m$

In [6]:
air_data = []
with open(os.path.join(base_path, "pressure.txt"), "r") as f:
    while True:
        try:
            P, d_n = map(int, f.readline().split())
            air_data.append((P, d_n))
        except:
            break

In [7]:
def cmHg_to_atm(pressure: float):
    return pressure/76

def get_constant(d_n: int, d_P: float):
    return d_P/d_n

In [8]:
L = 3              # air cell distance (cm)
wavelength = 632.8 # He-Ne wavelength (nm)
def air_refractive_index(wavelength, L, C, P=1):
    return (wavelength*1e-9/(2*L*1e-3*C))*P + 1

In [9]:
air_index = 1.000293  # theoretical value
indicies = []
for P, d_n in air_data:
    C = cmHg_to_atm(get_constant(d_n, P))
    index = air_refractive_index(wavelength, L, C)
    print(f"Air Refractive Index: {index}")
    indicies.append(index)
    print(f"Error Rate: {error_rate(air_index, index)}%")
    
mean_index = np.mean(indicies)
print(f"Mean Air Ref.: {mean_index:.5f}")
print(f"Error Rate: {error_rate(air_index, index)}%")

Air Refractive Index: 1.0032061866666666
Error Rate: 0.2912333352994084%
Air Refractive Index: 1.0032061866666666
Error Rate: 0.2912333352994084%
Air Refractive Index: 1.0032061866666666
Error Rate: 0.2912333352994084%
Air Refractive Index: 1.0032061866666666
Error Rate: 0.2912333352994084%
Air Refractive Index: 1.0032061866666666
Error Rate: 0.2912333352994084%
Mean Air Ref.: 1.00321
Error Rate: 0.2912333352994084%


## Exp 4: 유리판 굴절률
$n_g=\cfrac{n_a (1-cos⁡\theta)(2n_a t-N\lambda)}{2n_a t(1-cos⁡\theta)-N\lambda}$

In [10]:
glass_data = []
with open(os.path.join(base_path, "glass.txt"), "r") as f:
    while True:
        try:
            degree, d_n = map(int, f.readline().split())
            glass_data.append((degree, d_n))
        except:
            break
            
glass_data

[(2, 2), (4, 5), (6, 11), (8, 17), (10, 24)]

In [11]:
n_a = 1.0003              # air refractive index
t = 3e-3                  # glass thickness
wavelength = 632.8 * 1e-9 # He-Ne wavelength
def glass_refractive_index(n_a, degree, t, N, wavelength):
    nom = n_a*(1-np.cos(degree*np.pi/180))*(2*n_a*t - N*wavelength)
    denom = 2*n_a*t*(1-np.cos(degree*np.pi/180)) - N*wavelength
    return nom/denom

In [12]:
glass_index = 1.52
glass_indicies = []
for degree, d_n in glass_data:
    index = glass_refractive_index(n_a, degree, t, d_n, wavelength)
    glass_indicies.append(index)
    print(f"Glass Refractive Index: {index}")
    print(f"Error Rate: {error_rate(glass_index, index)}%")
    
mean_glass = np.mean(glass_indicies)
print(f"Glass Refractive Index: {mean_glass}")
print(f"Error Rate: {error_rate(glass_index, mean_glass)}%")

Glass Refractive Index: 1.5295575923956286
Error Rate: 0.6287889733966177%
Glass Refractive Index: 1.2758950267921005
Error Rate: 16.05953771104602%
Glass Refractive Index: 1.2674821926593556
Error Rate: 16.61301364083187%
Glass Refractive Index: 1.223925752064377
Error Rate: 19.478568943133098%
Glass Refractive Index: 1.197171233123431
Error Rate: 21.23873466293217%
Glass Refractive Index: 1.2988063594069785
Error Rate: 14.552213196909308%


# Fabry-Perot Interferometer
$2dcos\theta = m\lambda$,   m = 1, 2, 3 ...

In [13]:
actuator_distance

120.5460883445191

In [14]:
laser_wavelength = 632.8
fabry_data = []
with open(os.path.join(base_path, "fabry.txt"), "r") as f:
    while True:
        try:
            d_n, V = map(float, f.readline().split())
            fabry_data.append((d_n, V))
        except:
            break
fabry_data

[(3.0, 6.88), (6.0, 15.6), (9.0, 23.38), (12.0, 31.13), (15.0, 38.78)]

In [15]:
fabry_wav = []
for d_n, V in fabry_data:
    wav = get_wavelength(d_n, V, actuator_distance)
    fabry_wav.append(wav)
    print(f"Error Rate: {error_rate(laser_wavelength, wav)}%")
    
mean_fabry = np.mean(fabry_wav)
print(f"Mean Wavelength: {mean_fabry}nm")
print(f"Error Rate: {error_rate(laser_wavelength, mean_fabry)}")

Wavelength : 552.9047nm
Error Rate: 12.625675536210341%
Wavelength : 626.8397nm
Error Rate: 0.9418995904710158%
Wavelength : 626.3039nm
Error Rate: 1.0265646335560938%
Wavelength : 625.4333nm
Error Rate: 1.1641453285693342%
Wavelength : 623.3036nm
Error Rate: 1.5006888748324552%
Mean Wavelength: 610.9570425516182nm
Error Rate: 3.451794792727837


$y(x, t) = 2y_{max}sin(kx)cos(2 \pi ft)$